In [2]:
import numpy as np
import time
from numba import njit, float64, int32
from nonasymptotic.envs import StraightLine

In [3]:
d = 20 
env = StraightLine(d, 0.5, seed=1999)
conn_rad = 0.5
N = 100000

In [4]:
data = np.zeros((N, d))
for i in range(N):
    data[i, :] = env.sample_from_env()

In [5]:
# let's try to implement the hashgrid using ndarrays with adjacent memory locations.
# lookup is more expensive ~~ d * log n + d * N/R (search then intersect)

# but may be fast enough with enough optimization

@njit(int32[:, :](float64[:, :], float64))
def brute_force_ann(_data, _conn_rad):
    _N, _d = _data.shape
    normed_data = _data / _conn_rad

    sorted_args = [np.argsort(normed_data[:, _id]) for _id in range(_d)]
    sorted_coords = [normed_data[_sargs, _id] for _id, _sargs in enumerate(sorted_args)]

    # next, we do the lookup
    edge_list = []
    for _in in range(_N):
        # print('operating on: %d' % _in)
        pt = _data[_in]

        lb_sarg = np.searchsorted(sorted_coords[0], pt[0] - conn_rad, side='left')
        ub_sarg = np.searchsorted(sorted_coords[0], pt[0] + conn_rad, side='left')
        neighbors = sorted_args[0][lb_sarg:ub_sarg]
        
        for _id in range(1, _d):
            lb_sarg = np.searchsorted(sorted_coords[_id], pt[_id] - conn_rad, side='left')
            ub_sarg = np.searchsorted(sorted_coords[_id], pt[_id] + conn_rad, side='left')
            # neighbors = neighbors[np.in1d(neighbors, sorted_args[_id][lb_sarg:ub_sarg], assume_unique=True)]
            neighbors = np.intersect1d(neighbors, sorted_args[_id][lb_sarg:ub_sarg])
        
        edge_list.append(neighbors)
        
    return edge_list

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo conversion from list(array(int64, 1d, C))<iv=None> to array(int32, 2d, A) for '$270return_value.1', defined at None
[1m
File "../../../../../tmp/ipykernel_1093923/2170495332.py", line 32:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of assignment at /tmp/ipykernel_1093923/2170495332.py (32)[0m
[1m
File "../../../../../tmp/ipykernel_1093923/2170495332.py", line 32:[0m
[1m<source missing, REPL/exec in use?>[0m


In [5]:
st = time.perf_counter()
out = brute_force_ann(data, conn_rad)
et = time.perf_counter()
print(et - st)

print(max([len(_o) for _o in out]))
print(min([len(_o) for _o in out]))


0.07614191700122319


In [82]:
out

[array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),


In [23]:
def halve(arr):
    blah = arr / 2


a = np.ones(2)
halve(a)
a

array([1., 1.])